In [36]:
import numpy as np
import os
import json

Scannet_Root = "/home/yang/990Pro/scannet_seqs/data"
split = 'val'

scan_ids = os.listdir(os.path.join(Scannet_Root, split))

In [39]:
# get ref scans and reccans, scene_00xx_00, scene_00xx_01
room_scans = {}
num_scan = 0
for scan_id in scan_ids[:100]:
    room_id = scan_id.split('_')[0]
    scan_idx = scan_id.split('_')[1]
    if room_id not in room_scans:
        room_scans[room_id] = []
    room_scans[room_id].append(scan_id)
    num_scan += 1
print('num of room_ {}, with {} scans'.format(len(room_scans), num_scan))
# get rooms with multiple scan pairs
room_scan_multi = {}
for room_id, scans in room_scans.items():
    if len(scans) > 1:
        room_scan_multi[room_id] = scans
print('num of room_scan_multi:', len(room_scan_multi))

num of room_ 52, with 100 scans
num of room_scan_multi: 29


In [33]:
# get room scans with significant object difference
obj_diff_th = 5
scans_with_diff = {}

for room_id in room_scan_multi:
    ## get number of objects in each scan
    num_objects = {}
    for scan_id in room_scan_multi[room_id]:
        scan_path = os.path.join(Scannet_Root, split, scan_id, scan_id + "_vh_clean.aggregation.json")
        with open(scan_path) as f:
            annotation = json.load(f)
            num_objects[scan_id] = len(annotation['segGroups'])
    ## get scans with significant different number of objects
    scan_ids = list(num_objects.keys())
    num_objects = list(num_objects.values())
    diff = np.abs(np.array(num_objects)[:, None] - np.array(num_objects))
    diff = diff - np.diag(np.diag(diff))
    max_diff = np.max(diff)
    if max_diff > obj_diff_th:
        scan_ids = [scan_ids[np.argmax(diff) // len(scan_ids)], scan_ids[np.argmax(diff) % len(scan_ids)]]
        scans_with_diff[room_id] = scan_ids

In [40]:
room_scans

{'scene0011': ['scene0011_00', 'scene0011_01'],
 'scene0015': ['scene0015_00'],
 'scene0019': ['scene0019_00', 'scene0019_01'],
 'scene0025': ['scene0025_00', 'scene0025_01', 'scene0025_02'],
 'scene0030': ['scene0030_00', 'scene0030_01', 'scene0030_02'],
 'scene0046': ['scene0046_00', 'scene0046_01', 'scene0046_02'],
 'scene0050': ['scene0050_00', 'scene0050_01', 'scene0050_02'],
 'scene0064': ['scene0064_00', 'scene0064_01'],
 'scene0077': ['scene0077_00', 'scene0077_01'],
 'scene0081': ['scene0081_00', 'scene0081_01', 'scene0081_02'],
 'scene0084': ['scene0084_00', 'scene0084_01', 'scene0084_02'],
 'scene0086': ['scene0086_00', 'scene0086_01', 'scene0086_02'],
 'scene0088': ['scene0088_00', 'scene0088_01', 'scene0088_02', 'scene0088_03'],
 'scene0095': ['scene0095_01', 'scene0095_00'],
 'scene0100': ['scene0100_00', 'scene0100_01', 'scene0100_02'],
 'scene0131': ['scene0131_00', 'scene0131_01', 'scene0131_02'],
 'scene0139': ['scene0139_00'],
 'scene0144': ['scene0144_00', 'scene014

In [41]:
# save to file
import pickle
file_save = os.path.join(Scannet_Root, "files", 'scans_{}.pkl'.format(split))
pickle.dump(room_scans, open(file_save, "wb"))